# Regression Discontinuity Design

# 1. Data simulation: a predictor of the outcome

In [6]:
from numpy.random import normal, seed
from numpy import abs, cos, mean, pi, arange
import statsmodels.formula.api as smf
import pandas as pd
from rdrobust import rdrobust

seed(1234)

periods_n = 15
impact = 1000

time_points = arange(-periods_n, periods_n + 1)
n = len(time_points)
seasonality = -cos(time_points / (2 * pi))

D = (time_points >= 0).astype(int)
Y_0 = 500 + 100 * seasonality + normal(size=n, scale=50)
Y_1 = Y_0 + impact * D / (abs(time_points) + 1)
Y = D * Y_1 + (1 - D) * Y_0

# 2. Difference of means method

In [7]:
results = []
time_windows = arange(10, 1, -1)
for time_window in time_windows:
    ind = abs(time_points) < time_window
    results.append(mean(Y[ind & D == 1]) - mean(Y[ind & D == 0]))

pd.DataFrame({"time_window": time_windows, "estimate": results})

,time_window,estimate
0,10,221.475918
1,9,224.150524
2,8,236.824575
3,7,258.067470
4,6,295.477331
5,5,340.196316
6,4,421.326702
7,3,488.644188
8,2,617.177211


# 3. Linear models

In [8]:
d_coefs = []
lower_cis = []
upper_cis = []
time_windows = arange(10, 3, -1)
for time_window in time_windows:
  ind = abs(time_points) < time_window
  time_points_window = time_points[ind]
  Y_window = Y[ind]
  D_window = D[ind]
  df = pd.DataFrame({'y': Y_window, 'd': D_window, 'time_points': time_points_window})
  model = smf.ols(formula='y ~ d*time_points', data=df).fit()
  d_coefs.append(model.params['d'])
  lower_cis.append(model.conf_int().loc['d'][0])
  upper_cis.append(model.conf_int().loc['d'][1])

results = pd.DataFrame({
    'time_window': time_windows,
    'd': d_coefs,
    '2.5%': lower_cis,
    '97.5%': upper_cis
})
results

,time_window,d,2.5%,97.5%
0,10,653.111459,357.564054,948.658864
1,9,681.025774,373.307347,988.744200
2,8,730.194596,402.566327,1057.822865
3,7,773.285055,421.005356,1125.564754
4,6,885.936954,508.083285,1263.790622
5,5,948.844521,509.122373,1388.566668
6,4,970.354648,173.966523,1766.742774


# 4. Rdrobust library

In [9]:
rdrobust(Y, time_points)

Call: rdrobust
Number of Observations:                    31
Polynomial Order Est. (p):                  1
Polynomial Order Bias (q):                  2
Kernel:                            Triangular
Bandwidth Selection:                    mserd
Var-Cov Estimator:                         NN

                                Left      Right
------------------------------------------------
Number of Observations            15         16
Number of Unique Obs.             15         16
Number of Effective Obs.           5          6
Bandwidth Estimation           5.496      5.496
Bandwidth Bias                 8.188      8.188
rho (h/b)                      0.671      0.671

Method             Coef.     S.E.   t-stat    P>|t|       95% CI      
-------------------------------------------------------------------------
Conventional     952.305   351.14    2.712   6.687e-03 [264.083, 1640.526]
Robust                 -        -    2.598   9.368e-03 [255.742, 1826.121]
